# Poisson likelihood calculator

Given $M$ components, each with an estimated rate $\vec{\beta}$ determined by a normal distribution with uncertainty $\vec{\sigma}$, calculate the confidence itervals and perform a hypothesis tests for each parameter $b$.

Nominally each event corresponds to a set of observables $\vec{x}$ of $N$ measurements, for any given measurement, the probability for that particular measurement to come from a particular components is given by

$$ P_i(\vec{x}) \tag{1}$$

The prior probability is then formed through a combination of these components such that the total probability is 

$$ \mathbf{P} = \sum_i^M P_i(\vec{x}) \tag{2}$$

The likelihood for a full data set of $N$ measurements is the product of each event total probability

$$
\mathcal{L}(\vec{x}) = 
\prod_j^N \left( \sum_i^M b_iP_i(\vec{x}) \right) / \sum_i^Mb_i 
\tag{3}
$$

We can extend the likelihood by proclaiming that each components as well as the sum of components are simply a stochastic process, produces the extended likelihood:

$$
\mathcal{L}(\vec{x}) = 
\frac{\text{e}^{-\sum_i^Mb_i}}{N!} \prod_j^N \left( \sum_i^M b_iP_i(\vec{x}) \right) 
\tag{4}
$$

Finally, we can claim that we have _a priori_ knowledge of the parameters, whether it be through side-band analysis or external constraints, by including those constraints via some prior probability. Given no specific knowledge of the shape of that prior, we will consider the information we receive on the variables to be normally distributed and multiply the likelihood by those constraints

$$
\mathcal{L}(\vec{x}) = 
\frac{\text{e}^{-\sum_i^Mb_i}}{N!} 
\prod_j^N \left( \sum_i^M b_iP_i(\vec{x}) \right) 
\frac{1}{\sqrt{2\pi \sigma_i^2}}
\text{exp}\left({\frac{-(\beta_i-b_i)^2}{2\sigma_i^2}}\right)
\tag{5}
$$

A few definitions to simplify things:
$$ \lambda := \sum_i^Mb_i \tag{6}$$

Then then our objective function $\mathcal{O} = -\text{Ln}\mathcal{L}$

$$
\mathcal{O} = \lambda + \text{Ln}N! 
-\sum_j^N\text{Ln}\left( \sum_i^M b_iP_i(\vec{x}) \right) 
+ \sum_i^M \left( \frac{(\beta_i-b_i)^2}{2\sigma_i^2} 
    + \text{Ln}\sqrt{2\pi \sigma_i} \right)
\tag{7}
$$

Finally, for a counting analysis we assume that an optimal set of cuts has been applied which optimizes the sensitivity to a particular parameter, which simplifies the likelihood such that

$$ P_i(\vec{x}) := 1 \tag{8}$$

Also, because the shape of the likelihood space is independent of constant parameters, we can drop the $\text{Ln}\sqrt{2\pi \sigma_i}$ terms. We could also remove the $\text{Ln}N!$ term as well, but for numerical stability we will keep it around, but use Sterling's approximation: $\text{Ln}N! \approx N\text{Ln}N - N$. The remaining objective function we will thus use is:

$$
\mathcal{O} = \lambda - N\text{Ln}\lambda + N\text{Ln}N - N 
    + \sum_i^M \left( \frac{(\beta_i-b_i)^2}{2\sigma_i^2} \right)
\tag{9}
$$

_Note: If the different values of $\beta$ differ by orders of magnitude, it might be worth forming an affine invariant form of the likelihood, otherwise the $\text{Ln}\sqrt{2\pi \sigma_i}$ term should not matter_

[Profile Likelihood](https://www.stat.tamu.edu/~suhasini/teaching613/chapter3.pdf)

In [ ]:
push!(LOAD_PATH, "../src/")
using Batman

### Model Building
Three structures are used to build, test, and fit models. These are the `Model`, `Component`, and `Result`--all of which are mutable.

We begin by initializing a default `Model` and one-by-one use `add_component!` to modify the `Model`.

The intention is to provide multiple ways to evaluate the uncertainties, the default here is to compute the profile likelihood.

In [ ]:
m = CountingExperiment()

add_component!(m, "Signal", 20.0)
add_component!(m, "Bkg 1", 30.0; σ=0.1)
add_component!(m, "Bkg 2", 40.0; σ=45.0)
add_component!(m, "Bkg 3", 12.0; σ=12.0)

set_counts!(m, 100)

results = minimize!(m)

compute_profiled_uncertainties!(results; σ=1)
pretty_results(results)

In [ ]:
using PyPlot
plt.style.use("bat.mplstyle")

hs = x -> x >= 0 ? 1 : 0
#profile!("Signal", results; prior=nothing)
#uncertainty!("Signal", results )

interval_plot(results, "Signal")
plt.savefig("profile.svg")
plt.show()

In [ ]:
using PyPlot
correlation_plots(results)
plt.show()

In [ ]:
correlation_plots2(results)

In [ ]:
# Correlation Matrix -- Derive from correlation_plots, store in results

In [ ]:
# Generate fake experiments; two set => Positive, Negative

## Is this the correct way to express everything. One model per time t (say day)

days = collect(1:5:1000) # 1 to 100 days
days = exp10.(range(0,stop=4,length=100))
hs = x -> x >= 0 ? 1 : 0

results_set = []
for d in days
    m = CountingExperiment()

    add_component!(m, "Signal", 2.0*d)
    add_component!(m, "Bkg 1", 3.0*d; σ=0.2*d)

    set_counts!(m, 4.0 * d)

    results = minimize!(m)
    profile!("Signal", results; prior=hs, step=0.1, stop=20)
    uncertainty!("Signal", results, σ=1)   
    push!(results_set, results)
end

# Results stored in a DataFrame.jl
#compute_profiled_uncertainties!(results; σ=1)
#pretty_results(results)

In [ ]:
using SpecialFunctions
sens = []

function sensitivity(name, results;
    mode="FC", prior=nothing)

    comp = getparam(results.model, name)
    x, y = comp.likelihood_x, comp.likelihood_y
    if prior != nothing
        y = y .* prior.(x)
    end
    xselect = (x .>= 0)
    x = x[xselect]
    y = y[xselect]
    y = y / sum(y)
    cumulative = cumsum(y)
    if mode == "FC"
        cy = sortperm(y; rev=true)
        cum_y = cumsum(y[cy])
        cum_x = x[cy]
        # Need to find where it first crosses zero
        first_min = cum_y[ cum_x .== x[1] ]
    end
    return minimum([first_min[1], 1])
end

@show sensitivity("Signal", results_set[20])

for rs in results_set
    push!(sens, erfinv( sensitivity("Signal", rs))*2^0.5 )
end


plt.plot(days, 1/sqrt(4)*sqrt.(days),".")
plt.plot(days, 1/sqrt(3)*sqrt.(days),".")
plt.plot(days, sens)
plt.yscale("log")
plt.xscale("log")
plt.show()
#interval_plot(results_set[20], "Signal")
#@show days[ lb .>= 0 ][1]
#plot(days, lb)

In [ ]:
# Cowen Plot

In [ ]:
# ROC curve

In [ ]:
# Confusion Matrix

In [ ]:
# Documentation

In [ ]:
# Refactor code into sub-files, project.toml, mybinder, etc ...

In [ ]:
# Time dependent systematic (maybe)